In [15]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
import useful_functions as uf
from sklearn.preprocessing import OneHotEncoder
path_fill = '/workspaces/CMB_match/data/2022/filled_trian.csv'
path_train_pure = '/workspaces/CMB_match/data/2022/train.csv'
# data/2022/filled_trian.csv
df_train_filled = pd.read_csv(path_fill, index_col=0)
df_train = pd.read_csv(path_train_pure)

有序型分类变量：使用LabelEncoder()编码；

无序型分类变量：使用独热编码。

In [16]:
string_col =uf.dynamic_string_col(df_train_filled)

opt_col = string_col.copy()

df_str=pd.DataFrame()
if 'MON_12_CUST_CNT_PTY_ID' in string_col:
    # This a Binary Y/N, the training sample has Y only, do not fill with the mode
    opt_col.remove('MON_12_CUST_CNT_PTY_ID')
    df_str.loc[:,'MON_12_CUST_CNT_PTY_ID'] = df_train_filled['MON_12_CUST_CNT_PTY_ID']
    # Direct copy
# fill the missing value with mode
df_str.loc[:,opt_col] = uf.mode_nan_string(df_train_filled.loc[:, opt_col])
## One-hot encoding
df_str.loc[:,'SHH_BCK']=df_str.SHH_BCK.astype(str) # it was floated
dummies = pd.get_dummies(df_str,dummy_na=True)

In [17]:
_df_numerical= uf.get_numerical_df(df_train_filled)

1 无序分类变量的相关性
2 get the correlation matrix of the numerical variables
3 save the first column with the lowest missing value

In [18]:
corr_cols = uf.high_corr_col(df_train_filled)
drop_col = uf.save_n_drop(corr_cols,df_train_filled)
test = df_train_filled.drop(drop_col,axis=1)
second_corr = test.loc[:,uf.high_corr_col(test)].corr()
second_corr

,MON_12_EXT_SAM_TRSF_IN_AMT,EMP_NBR,LAST_12_MON_COR_DPS_DAY_AVG_BAL,CUR_MON_COR_DPS_MON_DAY_AVG_BAL,HLD_DMS_CCY_ACT_NBR
MON_12_EXT_SAM_TRSF_IN_AMT,1.000000,0.007209,0.448644,0.415210,0.055374
EMP_NBR,0.007209,1.000000,0.014451,0.089909,0.802149
LAST_12_MON_COR_DPS_DAY_AVG_BAL,0.448644,0.014451,1.000000,0.935051,0.087795
CUR_MON_COR_DPS_MON_DAY_AVG_BAL,0.415210,0.089909,0.935051,1.000000,0.143070
HLD_DMS_CCY_ACT_NBR,0.055374,0.802149,0.087795,0.143070,1.000000


In [19]:
rows,cols = np.where(second_corr>0.8)
# gives the position of all correlation value >0.8
get_rid_2=list()
for i in range(len(rows)):
    posi= second_corr.iloc[rows[i],cols[i]]
    # rows[i],cols[i] is the position of the element in the matrix
    # posi is the correlation value of rows[i] and cols[i]
    if posi<0.9990:
        col_set = {second_corr.index[rows[i]], second_corr.columns[cols[i]]}
        # this is the actual column name
        if col_set not in get_rid_2: # because the set disregard the order,
            # so we need to check if the set is already in the list,
            # the pair with the same two elements
            # but different order will be regarded as same set,
            get_rid_2.append(col_set)
drop_col_2  = list(itertools.chain(*get_rid_2)) # take component out to list
# drop_col_2 is the list of column name that need to be dropped

,CUST_UID,LABEL,AGN_CNT_RCT_12_MON,ICO_CUR_MON_ACM_TRX_TM,NB_RCT_3_MON_LGN_TMS_AGV,AGN_CUR_YEAR_AMT,AGN_CUR_YEAR_WAG_AMT,AGN_AGR_LATEST_AGN_AMT,ICO_CUR_MON_ACM_TRX_AMT,COUNTER_CUR_YEAR_CNT_AMT,...,HLD_DMS_CCY_ACT_NBR,REG_DT,OPN_TM,HLD_FGN_CCY_ACT_NBR,NB_CTC_HLD_IDV_AIO_CARD_SITU,AI_STAR_SCO,LGP_HLD_CARD_LVL,MON_12_CUST_CNT_PTY_ID,WTHR_OPN_ONL_ICO,SHH_BCK
0,2a171d461bf24a739b804c2843ad4f6e,0,-1.560883e+05,70.000000,423.300000,-1.817677e+09,5.047545e+08,1.010124e+08,2.587998e+07,0.000000e+00,...,10.0,1706.450000,414.84,0.0,NaN,5.0,NaN,NaN,B,30.0
1,c58d5848d18548e297963c2d9e092699,0,2.280000e+03,220.000000,783.131874,1.177657e+07,4.412362e+08,1.964624e+06,1.475550e+07,0.000000e+00,...,20.0,197.420000,193.87,0.0,D,5.0,F,Y,B,30.0
2,74e6d8179e784a039bcb5a722014f4a4,0,-1.089318e+06,0.000000,-156.311311,-4.598293e+08,3.130245e+08,4.599820e+06,0.000000e+00,0.000000e+00,...,10.0,1133.550000,1120.00,0.0,NaN,5.0,F,NaN,B,0.0
3,1b6a8be2c2e34de09ca00d71470ec180,1,-5.782936e+05,-7738.076281,20.000000,2.970520e+09,2.218253e+09,-7.100510e+07,4.699300e+09,0.000000e+00,...,10.0,96.450000,90.00,0.0,C,1.0,NaN,NaN,A,10.0
4,1974b5e0440a41128a416a8cdc8a7c16,1,-8.771333e+05,-19616.415412,336.991623,1.168100e+09,3.291222e+09,-1.558762e+08,1.470712e+10,-2.670062e+06,...,10.0,6450.072937,353.87,0.0,C,4.0,NaN,Y,A,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,ca299adfc4be4af0b11c85abf28a12e6,0,-1.318000e+06,0.000000,53.300000,-8.382024e+08,-8.921546e+08,1.123244e+08,0.000000e+00,0.000000e+00,...,20.0,202.260000,128.06,0.0,B,2.0,C,NaN,B,10.0
39996,c13f097515ee47ca962526482ab500fb,0,-3.672142e+05,70.000000,143.300000,-6.434461e+06,1.952919e+09,-6.142260e+07,1.643146e+07,0.000000e+00,...,10.0,174.520000,158.71,0.0,NaN,1.0,NaN,NaN,B,10.0
39997,b8ec8daa79234cd1aceed066bf78f069,0,2.153000e+04,1130.000000,723.259154,8.102717e+07,-7.369833e+08,1.277221e+07,5.962545e+08,0.000000e+00,...,10.0,2263.550000,2130.65,0.0,NaN,5.0,D,Y,B,30.0
39998,1a538dd9745249b7b9d1f52b6edb80ba,1,7.136730e+05,10.000000,126.700000,-2.021357e+09,1.484786e+09,-2.141534e+08,4.885000e+05,0.000000e+00,...,10.0,514.520000,508.06,0.0,NaN,2.0,NaN,NaN,B,10.0


4 use the drop_col we get to drop again

In [20]:
uf.save_n_drop(drop_col_2,test)

[]